# Predict Weather: Temperature Forecast

With this Jupyter notebook you'll walk through the necessary steps to create your first Azure ML pipeline to preprocess data, train a ML model, and register it. Additionally, you'll deploy this model to the cloud to obtain a REST endpoint for predictions.

![image info](./images/tutorial-outline.png)

Our dataset of choice is the freely available daily weather data from DWD. A description of the data itself is available [here](https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate/daily/kl/historical/DESCRIPTION_obsgermany_climate_daily_kl_historical_en.pdf) (English), or alternatively [here](https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate/daily/kl/historical/BESCHREIBUNG_obsgermany_climate_daily_kl_historical_de.pdf) in German.

**Our objective is to predict the average air temperature for the next day based on historical data.** For this we will build a type of recurrent neural network called Long-Short Term Memory (LSTM) using TensorFlow.

Note that some datasets are already available in our Azure Machine Learning Workspace. Just take a look which data is available and use the one that suits you.

# Notebook Setup

Fill in or modify these variables according to your needs. 

In the case of datasets, refer to the Azure ML workspace and see which datasets prefixed with `weather` are available.

In [ ]:
initials = ""  # TODO
cluster_name = ""  # TODO

data_name = "weather-nurnberg"


## Connect to an Azure ML Workspace

Recall that a Azure ML Workspace manages every resource and asset relatd to Machine Learning such as datasets, models, experiment with their runs, and pipelines with their runs.

![image info](./images/azure-machine-learning-taxonomy.png)

Connection to a Azure ML (AML) workspace can be accomplished in one of two ways: 

1. Either from a configuration file, which is downloadable from the Web-UI of the AML workspace itself: 
   ``` python
   ws = Workspace.from_config()
   ```
2. Connect to workspace by specifying the appropriate parameters:
   ``` python
   ws = Workspace.get(
      name='aml-workspace',
      subscription_id='1234567-abcde-890-fgh...',
      resource_group='aml-resources'
   )
   ```



In [ ]:
from azureml.core import Workspace

# connect to workspace from configuration file
ws = Workspace.from_config()

print(
    "Workspace name: " + ws.name,
    "Azure region: " + ws.location,
    "Subscription id: " + ws.subscription_id,
    "Resource group: " + ws.resource_group,
    sep="\n",
)

# Training Setup

## Create Experiment

According to Merriam-Webster, an experiment is defined as "an operation or procedure carried out under controlled conditions in order to discover an unknown effect or law, to test or establish a hypothesis, or to illustrate a known law". With Azure Machine Learning you can manage experiments and different experiment *runs* in an organized manner. Thus, for our weather prediction project, we as well create a dedicated experiment as follows:

In [ ]:
from azureml.core import Experiment

experiment = Experiment(workspace=ws, name="ex-weather-predict-" + initials)


## Create or Use a Compute Cluster

Our training pipeline will eventually run somewhere. Two options are available: On your local device, or on a specified compute cluster. In our case today we use a compute cluster. In the following Python cell we first try to get an existing cluster with a specified name. If it is not available yet, we simply create it. In any case, we will wait until the minimum nodes are provided.

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Check if the compute target exists
try:
    cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print("Found existing cluster.")
except ComputeTargetException:
    # If not, create it
    compute_config = AmlCompute.provisioning_configuration(
        vm_size="STANDARD_DS11_V2",
        min_nodes=0,  # Default value is 0. Anyways, defining zero compute nodes keeps cost at a minimum.
        max_nodes=2,
    )
    cluster = ComputeTarget.create(ws, cluster_name, compute_config)

cluster.wait_for_completion(show_output=True)

## Create a Python Environment 

Additionally, we need to specify a Python environment. In our case, we will create a new one that satisfies our needs. That is: scikit-learn, pandas, numpy, joblib, and tensorflow.

In [ ]:
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.runconfig import RunConfiguration
from azureml.core.runconfig import DockerConfiguration

env_name = "weather-predict-" + initials

environment = Environment(env_name)
environment.python.user_managed_dependencies = False  # Let Azure ML manage dependencies

# Create a set of package dependencies
python_packages = CondaDependencies.create(
    python_version="3.8",
    conda_packages=["scikit-learn", "pandas", "numpy", "joblib", "tensorflow"],
    pip_packages=["azureml-sdk", "azureml-defaults"],
)

# Add the dependencies to the environment
environment.python.conda_dependencies = python_packages

# Register the environment (just in case you want to use it again)
environment.register(workspace=ws)
registered_env = Environment.get(ws, env_name)

# Create a new runconfig object for the pipeline
# RunConfiguration encapsulates necessary information for submitting training runs
# to different compute targets.
pipeline_run_config = RunConfiguration()

# Use the compute you created above.
pipeline_run_config.target = cluster

# Run within a docker container
pipeline_run_config.docker = DockerConfiguration(use_docker=True)

# Assign the environment to the run configuration
pipeline_run_config.environment = registered_env

print("Run configuration created.")


You can also retrieve all existing environments, i.e. all environments created and provided by Azure as well as custom ones:

In [ ]:
from azureml.core import Environment

environments = Environment.list(workspace=ws)
for name in environments:
    print("Name:", name)


## Create a Training Pipeline

Now we'll get to the meat: Creating a training pipeline consisting in the following steps:
1. **Preprocess Data**: This step will obtain a dataset name, retrieves the corresponding registered dataset, performs data cleaning, and normalization of input features. It then stores this preprocessed data and normalization information (transformer values) to Azure Storage. This normalization information is then published as a "dataset" because we will need it later in a scoring script. A script that receives user data, performs the same data preprocessing and feature normalization in order to retrieve predictions.
2. **Train Model**: This step trains a model based on the preprocessed data from the previous step. Additionally, we have several (user) arguments to change or specify the neural networks architecture or even select the type of layer. Currently, only Long-Short Term Memory (LSTM) is implemented. However, you may as well implement other types of network layers as you wish. For instance you could experiment with Gated Recurrent Units (GRU), or simple Recurrent Neural Network (RNN) layers. Whatever the (hyper-)parameters, this step will save any trained model to Azure Storage.
3. **Register Model**: This step picks up the previously persistet neural network model and registers it to Azure Machine Learning Workspace. After this step this model can be deployed.

For an introduction to LSTMs refer to this [blog](https://colah.github.io/posts/2015-08-Understanding-LSTMs/).

In [ ]:
from azureml.data import OutputFileDatasetConfig
from azureml.pipeline.steps import PythonScriptStep
from azureml.pipeline.core import PipelineData
from azureml.core import Dataset

import os

# To move data between pipeline steps, we first need to define data "sinks".
# This can be done with PipelineData or OutputFileDatasetConfig. We'll do
# both to demonstrate the difference between these two:

# Define a place where our preprocessed data will temporarily live
prepped_data = OutputFileDatasetConfig("prepped_" + initials)

# Get the default datastore
data_store = ws.get_default_datastore()
# Define a place where the transformer lives. We can also specify a exact destination with a string template:
transformer_data = OutputFileDatasetConfig(
    "transformer_" + initials, destination=(data_store, "transformerout/{run-id}")
)

# Registers min-max scaling values of column transformer as a new dataset after a specific pipeline has run successfully
# (we associate this data sink later).
# In case where such a dataset already exits, a new version is created. This will be helpful in our scoring script.
transformer_data_name = f"{data_name}-transformer-{initials}"
output_data_dataset = transformer_data.as_upload(overwrite=True).register_on_complete(
    name=transformer_data_name,
    description="Column transformer values for min-max scaling of features",
)
print(
    f"Min-max scaling values will be registered as a dataset under the name of: {transformer_data_name}"
)

# Create a place where our trained model will be stored
model_folder = PipelineData("model_folder_" + initials, datastore=data_store)

# Our model needs a name
model_name = f"{data_name}-model-{initials}"

# Our python scripts live somewhere. See the directory for detailed information.
src_dir = "src"


In [ ]:
# Step to run a Python script
preprocess = PythonScriptStep(
    name="Prepare Data",
    source_directory=src_dir,
    script_name="preprocess/main.py",
    compute_target=cluster,
    # Script arguments include PipelineData
    arguments=[
        "--dataset_name",
        data_name,
        "--output_folder",
        prepped_data,  # here we have the data sink for our preprocessed data
        "--transformer_folder",
        output_data_dataset,  # here we have the location for our transformer
        # you can also play with splitting ratios. Take a look at the script how the arguments are called
    ],
    runconfig=pipeline_run_config,  # associate the pipeline configuration we previously defined
    allow_reuse=True,  # Very useful. When input parameters did not change, this step will not be rebuild. Saves time - and money
)
print("Successfully created data preparation step.")


Play with the training parameters. Specifically, set the following parameters:
- `sequence_length`
- `batch_size`
- `epochs`
- `patience`
- `recurrent_units`: 
- `recurrent_model_type`: Implement GRU and/or RNN.

Keep the following in mind regarding sequence length: Data is prepared for the network such that it has the following shape: `(batch_size, sequence_length, features)`. Our target will be the next day's average air temperature. Following image illustrates one data sequence with sequence of 5 and a number of features of 3. With a batch size of 8, we will feed eight such windows with the corresponding sequence length to the model for training.

![image info](./images/data-reshaping.png)

In [ ]:
# Used as input arguments to the training script.
# Play with sequence length.
sequence_length = None  # TODO
batch_size = None  # TODO play with batch size
epochs = None  # TODO play with number of epochs

# play with early stopping. This will stop if now change in val loss is registered for a specified number of epochs
patience = None  # TODO

# play with number of recurrent units. Keep in mind that it is generally good to have lesser units than input sequence length
recurrent_units = None  # TODO

# go ahead and implement other layers
recurrent_model_type = "LSTM"  # TODO

# play with learning rate. In the case for Adam or Nadam, a learning rate of 0.001 is a good start
learning_rate = 0.001

# care to implement and use a different optimizer?
optimizer = "nadam"


train = PythonScriptStep(
    name="Train Model",
    source_directory=src_dir,
    script_name="train/main.py",
    compute_target=cluster,
    # Pass as script argument
    arguments=[
        "--input_folder",
        prepped_data.as_input(),  # here we specify out preprocessind folder as input so that our script can read data from it
        "--output_folder",
        model_folder,  # here we have the PipelineData where our model will be stored
        "--model_name",
        model_name,
        "--data_name",
        data_name,
        "--sequence_length",
        sequence_length,
        "--learning_rate",
        learning_rate,
        "--batch_size",
        batch_size,
        "--epochs",
        epochs,
        "--recurrent_model_type",
        recurrent_model_type,
        "--recurrent_units",
        recurrent_units,
        "--optimizer_name",
        optimizer,
        "--early_stopping_patience",
        patience,
    ],
    outputs=[
        model_folder
    ],  # Note that any PipelineData additionally need to be specified as output, and as input if it is used as an input
    runconfig=pipeline_run_config,
    allow_reuse=True,
)
print("Successfully created model training step.")


In [ ]:
register = PythonScriptStep(
    name="Register Model",
    source_directory=src_dir,
    script_name="register/main.py",
    arguments=[
        "--input_folder",
        model_folder,  # specify our model folder to read from
        "--model_name",
        model_name,  # our saved model will have a specific name
        "--register_model_name",
        model_name,  # we can also specify a different name to register it to Azure
    ],
    inputs=[
        model_folder
    ],  # here our PipelineData is used as an input, thus we also need to specify it as an input. Quite cumbersome, isn't it?
    compute_target=cluster,
    runconfig=pipeline_run_config,
    allow_reuse=False,
)
print("Successfully created model registration step.")


After specifying each step, we can build our Azure ML pipeline and submit it, i. e. execute those specified steps.

In [ ]:
from azureml.pipeline.core import Pipeline
from azureml.core import Experiment

# Construct the pipeline
train_pipeline = Pipeline(workspace=ws, steps=[preprocess, train, register])

# run the pipeline
run = experiment.submit(train_pipeline)
run.wait_for_completion()

### Show Run Metrics

We can take a look at metrics like this:

In [ ]:
from azureml.widgets import RunDetails

for step_run in run.get_children():
    print("{}: {}".format(step_run.name, step_run.get_metrics()))


Let's plot the training history:

In [ ]:
import matplotlib.pyplot as plt


def plot_history(history):
    logged_metrics = [
        metric for metric in history.keys() if not metric.startswith("val")
    ]

    for logged_metric in logged_metrics:
        plt.plot(history[logged_metric])
        plt.plot(history["val_" + logged_metric])
        plt.title(f"Learning curve for: {logged_metric.upper()}")
        plt.ylabel(logged_metric)
        plt.xlabel("epoch")
        plt.legend(["train", "val"], loc="upper left")
        plt.show()

In [ ]:
for step_run in run.get_children():
    if step_run.name == "Train Model":
        plot_history(step_run.get_metrics())

Alternatively, you can view them in Azure Machine Learning workspace under your experiment or specific run. See where you can find your metrics in Azure Machine Lerning Workspace browser. You can also add new plots to your experiment page like in the following screenshot:

![image info](./images/experiment-metrics.png)

## List all  Registerd Models

In [ ]:
from azureml.core import Model

for model in Model.list(ws):
    print(model.name, "version:", model.version)


# Deployment

We successfully created a training pipeline, run this pipeline and produced a model. Now we want to deploy it. Before we deploy it to an Azure Container Instance (ACI) we will deploy it locally. This is good practice as we can debug more easily. Let's see what we need for deployment:
1. Python Environment.
2. In our case we need to download the transformer dataset so that we can scale our features accordingly.
3. Inference configuration that points to our scoring script.
4. Deployment configuration. For local deployment this will be a local web service, for cloud deployment to an ACI.

## Define or Reuse an Inference Environment

A deployment configuration needs a python envronment. We try to reuse our previously created environment, but we can also specify a new one.

In [ ]:
try:
    registered_env = Environment.get(workspace=ws, name=env_name)
except:
    print("Environment not found, will create it.")

    inference_environment = Environment(env_name)
    inference_environment.python.user_managed_dependencies = (
        False  # Let Azure ML manage dependencies
    )

    # Create a set of package dependencies
    python_packages = CondaDependencies.create(
        python_version="3.8",
        conda_packages=["scikit-learn", "pandas", "numpy", "joblib", "tensorflow"],
        pip_packages=["azureml-sdk", "azureml-defaults"],
    )

    # Add the dependencies to the environment
    environment.python.conda_dependencies = python_packages

    # Register the environment (just in case you want to use it again)
    environment.register(workspace=ws)
    registered_env = Environment.get(ws, env_name)
else:
    print(f'Using existing environment named "{env_name}"')

## Get Column Transformer from Datastore

In our preprocessing step we not only cleaned the data, but also scaled the features. We stored these normalization information and now we need to download them to be accessible for our scoring script.

**TODO: Please specify the same sequence length in the scoring script as defined above for model training. (see `src/score/main.py`)**

In [ ]:
from azureml.core import Dataset


dataset = Dataset.get_by_name(ws, name=transformer_data_name)
dataset.download(target_path=os.path.join(src_dir, "score"), overwrite=True)


You might be wondering why we do not need to download the model itself. We will get the model information later. But, we only need the reference and not download it. This is because Azure already mounts the model into our docker container for us. Why downloading then the transformer information? We cannot access our Azure Machine Learning Workspace from within our scoring script. Well, we could, but we then would need to place our credentials into our docker container. This is not recommended.

## Deploy Model to Local  Webservice

### Define an Inference Configuration

Like our pipeline steps holds information to our scripts, an inference configuration does the same. It points to the right python environment, source directory that is then mounted into the docker container and specifies an entry step. 

In [ ]:
import os
from azureml.core import Environment
from azureml.core.model import InferenceConfig

src_dir = "src"

dummy_inference_config = InferenceConfig(
    environment=registered_env,  # we will reuse our training environment
    source_directory=src_dir,
    entry_script="score/main.py",
)

### Define a Deployment Configuration

We first deploy locally. Therefore, we specify a `LocalWebservice`:

In [ ]:
from azureml.core.webservice import LocalWebservice

local_deployment_config = LocalWebservice.deploy_configuration(port=6789)

### Get Your Model

Previously, in the registration step, we published our trained model such that Azure Machine Learninig knows about it and we can reuse it. Note that every model is associated with its own version number. This provides the ability to specify a model by its name, i. e. pin a model used for deployment to a predefined version. Here we simply use the lates version by not specifying a version number.

In [ ]:
from azureml.core.model import Model

model = Model(ws, model_name)
model

### Deploy Model to Local Webservice

At last, we can deploy locally. Simply hit `Model.deploy` and specify every other element we created (inference configuration, deployment configuration, model)

In [ ]:
service_name = "weather-predict-service-" + initials
service = Model.deploy(
    workspace=ws,
    name=service_name,
    models=[model],
    inference_config=dummy_inference_config,
    deployment_config=local_deployment_config,
    overwrite=True,  # Overwrite if there is already a local webservice with the same port running
)
service.wait_for_deployment(show_output=True)

In the case where you already deployed your model but did run into problems, you can reload your fixed service with the following code. No need to re-deploy again, simply reload your webservice.

In [ ]:
service.reload()

### Get Your Dataset For Prediction

Before calling the webservice, we need data. Thus, we get the same dataset but specify different samples. In our case here we use the last 60 days for which we want a prediction. (Keep in mind that we get fewer predictions depending on your sequence length)

In [ ]:
from azureml.core import Dataset

dataset = Dataset.get_by_name(ws, name="weather-nurnberg")
df = dataset.to_pandas_dataframe()

# take last 60 days
to_predict = df.iloc[-180:, :]
to_predict.head()

### Call Into Your Local Webserivce

Now we can actually call the REST-service that has been provided for us:

In [ ]:
import requests
import json

uri = service.scoring_uri
requests.get("http://localhost:6789")
headers = {"Content-Type": "application/json"}
data = json.dumps({"data": to_predict.to_dict("records")})
response = requests.post(uri, data=data, headers=headers)
print(response)
print(response.json())

Was your call successful? Let's check the logs with the following command:

In [ ]:
print(service.get_logs())

## Deploy to Azure Container Instance

After successful local deployment and testing we can deploy our service to the cloud. For this we need the same steps as with local deployment except for the `LocalWebservice`, which we will change to `AciWebservice`:

In [ ]:
from azureml.core.webservice import AciWebservice


crml_env = Environment.get(workspace=ws, name=env_name)

inference_config = InferenceConfig(
    environment=crml_env,
    source_directory=src_dir,
    entry_script="score/main.py",
)

deployment_config = AciWebservice.deploy_configuration(
    cpu_cores=0.5,  # Small amount of GPU is sufficient for our needs here
    memory_gb=1,  # Same goes for RAM. Our model and data (typically) are not large
    auth_enabled=True,  # Specify authentication with True
)

cloud_service = Model.deploy(
    ws,
    service_name,
    [model],
    inference_config,
    deployment_config,
    overwrite=True,
)
cloud_service.wait_for_deployment(show_output=True)

In [ ]:
print(cloud_service.get_logs())

### Call Into Your Remote Webservice

In [ ]:
import requests
import json
from azureml.core import Webservice

deployed_cloud_service = Webservice(workspace=ws, name=service_name)
scoring_uri = deployed_cloud_service.scoring_uri

# If the service is authenticated, set the key or token
key, _ = deployed_cloud_service.get_keys()

# Set the appropriate headers
headers = {"Content-Type": "application/json"}
headers["Authorization"] = f"Bearer {key}"

data = json.dumps({"data": to_predict.to_dict("records")})
response = requests.post(scoring_uri, data=data, headers=headers)
print(response.json())

In [ ]:
import pandas as pd

predictions = pd.DataFrame.from_records(response.json()["result"])
predictions.head()

# Is Your Prediction Any Good?

## Plot True vs. Predicted

In [ ]:
import matplotlib.pyplot as plt
import statsmodels.tsa.api as smt
import seaborn as sns
import numpy as np

In [ ]:
def plot_prediction(true, predicted, label_true="true", label_predicted="predicted"):
    # x = list(range(len(predicted)))
    x = true.index
    plt.plot(x, true, label=label_true)
    plt.plot(x, predicted, label=label_predicted)
    plt.xticks(rotation=45)
    plt.legend()
    plt.show()


In [ ]:
plot_prediction(to_predict.iloc[sequence_length:-1, -1], predictions.iloc[:, -1])

In [ ]:
from sklearn.metrics import mean_squared_error

rmse = mean_squared_error(
    to_predict.iloc[sequence_length:-1, -1], predictions.iloc[:, -1], squared=True
)
mse = mean_squared_error(
    to_predict.iloc[sequence_length:-1, -1], predictions.iloc[:, -1], squared=False
)

print(f"RMSE: {rmse}\nMSE: {mse}")


## Residual Analysis

According to statistician and researcher [Rob J. Hyndman](https://otexts.com/fpp3/diagnostics.html), good predictions have the following properties when taking a closer look at its residuals, i. e. errors (true minus predicted values):
1. Residuals are uncorrelated. Correlation means that there is still unused information. Using these information generally result in a better model.
2. Residuals have zero mean. Residuals with no zero mean indicate bias in the model.

Additionally, the following properties can be helpful as well although they are not necessary:
1. Residuals have constant variance ("homoscedasticity").
2. Residuals are normally distributed.

In [ ]:
def residual_analysis(y, lags=None, figsize=(10, 8)):
    fig = plt.figure(figsize=figsize)
    layout = (2, 2)
    ts_ax = plt.subplot2grid(layout, (0, 0), colspan=2)
    acf_ax = plt.subplot2grid(layout, (1, 0))
    hist_ax = plt.subplot2grid(layout, (1, 1))
    sns.lineplot(x=list(range(len(y))), y=y, ax=ts_ax)

    smt.graphics.plot_acf(y, lags=lags, ax=acf_ax)
    sns.distplot(y, ax=hist_ax)
    acf_ax.set_xlim(1.5)
    # acf_ax.set_ylim(-0.1, 0.2)
    fig.suptitle(f"Residual Analysis, Residual mean: {np.mean(y):.4f}")
    sns.despine()
    plt.tight_layout()
    plt.show()


In [ ]:
true = to_predict.iloc[sequence_length:-1, -1].to_numpy()
pred = predictions.iloc[:, -1].to_numpy()
residuals = true - pred
residuals.shape
residual_analysis(residuals)

# Clean Up

- Manually delete not needed experiment runs or the whole experiment itself.


In [ ]:
# delete deployed webservice
deployed_cloud_service.delete()

# Next Steps?
Our pipeline-code so far seems production ready, we specified all the needed steps and even deployed to an ACI. But are we really ready to deploy to production and go home?

Here are a few things to consider:
- Pipeline creation and submission was orchestrated from within this very Jupyter Notebook. This may not be good for an automated deployment.
- What if data changed? As of now, we would need to manually register this new data to our Azure Machine Learning Workspace and execute this Jupyter Notebook.
- Of course we could publish our Azure ML pipeline, but this only gives us a REST endpoint to trigger the execution of data preprocessing, model training, and model registration. 
- We may not want every pipeline excution to register a new model if the new model performs poorly compared to the already registered/deployed model. Thus, we may want an evaluation step that decides that for us.
